In [1]:
# @title NoSQL
# ---------------------------------------------------------------
#                    Librerías
# ---------------------------------------------------------------
import ipywidgets as widgets
from IPython.display import display
from pymongo import MongoClient
import time
import pandas as pd
import numpy as np
import os
from bayes_classifier import BayesianClassifier
from bayes_classifier import available_hypotheses
import matplotlib.pyplot as plt
import networkx as nx


# ---------------------------------------------------------------
#               Variables Globales
# ---------------------------------------------------------------
time_start = time.time()
classifier = BayesianClassifier()
print(f'Initializing BayesianClassifier: {time.time() - time_start:.4f}s')

# Get the transactions collection
transactions_col = classifier.db["transactions"]

output = widgets.Output()

graph_output = widgets.Output()

# ---------------------------------------------------------------
#             Componentes de Interfaz Gráfica
# ---------------------------------------------------------------
# ----- Texto: Instrucciones acerca del programa
INSTRUCCIONES_DEL_PROGRAMA = widgets.HTML("""
        <h1> Inferencia de fraude en transacciones de tarjetas de crédito
        </h1>
        <h2>Integrantes:</h2>
        <ul>
          <li>Daniel Sánchez (20130495)</li>
          <li>Javier Monzón (20121248)</li>
        </ul>
  """)

# ----- Texto: Instrucciones para la pestaña 'Clasificar'
INSTRUCCIONES_PESTANA1 = widgets.HTML("""
        <h3>Instrucciones:</h3>
        En esta pestaña podrás inferir si se cometió fraude o no para determinada transacción de tarjeta de crédito.<br>
        Para esto deberás seleccionar entre distintos valores posibles sobre la transacción.<br>
        También deberás seleccionar la hipótesis usada en la inferencia.<br>
        Haciendo click en 'Clasificar' podrás inferir si se cometió o no fraude.
  """)

# ----- Botón: Hipotesis
DROPDOWN_HYPOTHESIS_1 = widgets.Dropdown(options=list(
    available_hypotheses.items()), description='Hypothesis:')

DROPDOWN_HYPOTHESIS_2 = widgets.Dropdown(options=list(
    available_hypotheses.items()), description='Hypothesis:')
# ----- Botones: Evidencia

fields = ["age", "gender", "category", "amount_bin"]
distinct_values = {f: sorted(transactions_col.distinct(f)) for f in fields}
DROPDOWN_GENDER = widgets.Dropdown(
    options=distinct_values['gender'], description='Gender:')
DROPDOWN_AGE = widgets.Dropdown(
    options=distinct_values['age'], description='Age:')
DROPDOWN_CATEGORY = widgets.Dropdown(
    options=distinct_values['category'], description='Category:')
DROPDOWN_AMOUNT = widgets.Dropdown(
    options=distinct_values['amount_bin'], description='Amount:')

# ----- Botón: Clasificar
BOTON_CLASIFICAR = widgets.Button(
    description='Clasificar',
    tooltip='Clasificar',
    button_style='info',
    icon='check'
)

# ----- Valid: Resultado
VALID_RESULTADO = widgets.Valid(
    value=False,
    description='Fraud?',
)

# ----- Texto: Instrucciones para la pestaña 'Hipótesis'
INSTRUCCIONES_PESTANA2 = widgets.HTML("""
        <h3>Instrucciones:</h3>
        En esta pestaña podrás visualizar la hipótesis de la inferencia Bayesiana.
  """)

# ----- Función para construir pestañas


def construir_pestanas():
    pestanas_programa = widgets.Tab()
    cont_pestana1 = widgets.VBox(
        [INSTRUCCIONES_PESTANA1, DROPDOWN_HYPOTHESIS_1, DROPDOWN_GENDER, DROPDOWN_AGE, DROPDOWN_CATEGORY, DROPDOWN_AMOUNT, BOTON_CLASIFICAR, VALID_RESULTADO])
    cont_pestana2 = widgets.VBox(
        [INSTRUCCIONES_PESTANA2, DROPDOWN_HYPOTHESIS_2, graph_output])
    # ingresamos los componentes para cada pestaña
    pestanas_programa.children = [cont_pestana1, cont_pestana2]

    # Los nombres de cada pestaña
    pestanas_programa.set_title(0, 'Inferencia')
    pestanas_programa.set_title(1, 'Hipótesis')
    return pestanas_programa


# ----- Pestañas: son 2 pestañas cada una conteniendo botones
PESTANAS_DEL_PROGRAMA = construir_pestanas()

# ---------------------------------------------------------------
#             Gestión de Eventos
# ---------------------------------------------------------------
# --- Evento: Clasificar


def evento_clasificar(b):
    evidencia = {}
    evidencia['gender'] = DROPDOWN_GENDER.value
    evidencia['age'] = DROPDOWN_AGE.value
    evidencia['category'] = DROPDOWN_CATEGORY.value
    evidencia['amount_bin'] = DROPDOWN_AMOUNT.value
    with output:
        print(evidencia)
        res = classifier.classify(evidencia)
        print(res)
    VALID_RESULTADO.value = res[0]


def dropdown_hypothesis_1_event_handler(change):
    classifier.set_hypothesis(change.new)


def draw_graph(hypothesis):
    G = nx.DiGraph()
    for parent, children in hypothesis.items():
        for child in children:
            G.add_edge(parent, child)

    plt.figure(figsize=(6, 4))
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, arrows=True, node_color='lightblue',
            node_size=1500, font_size=10, edge_color='gray')
    plt.title("Estructura de Hipótesis")
    plt.tight_layout()
    return plt


def dropdown_hypothesis_2_event_handler(change):
    hypothesis = change.new
    with graph_output:
        graph_output.clear_output(wait=True)
        fig = draw_graph(hypothesis)
        fig.show()


# ---------------------------------------------------------------
#             Asignación de Evento a Componentes GUI
# ---------------------------------------------------------------
BOTON_CLASIFICAR.on_click(evento_clasificar)

DROPDOWN_HYPOTHESIS_1.observe(
    dropdown_hypothesis_1_event_handler, names='value')

DROPDOWN_HYPOTHESIS_2.observe(
    dropdown_hypothesis_2_event_handler, names='value')

# ---------------------------------------------------------------
#             GUI Final
# ---------------------------------------------------------------


def diseno_componentes():
    display(widgets.VBox(
        children=[INSTRUCCIONES_DEL_PROGRAMA, PESTANAS_DEL_PROGRAMA]), output)


# ---------------------------------------------------------------
#             Lanzamiento de la aplicación
# ---------------------------------------------------------------
diseno_componentes()

Changed hypothesis to: defaultdict(<class 'list'>, {})
Initializing BayesianClassifier: 1.6839s


Output()